# Partial Derivatives Computation via Autodiff

In this notebook, we show how to use the autodiff feature of $\partial\textrm{SGP4}$. Due to the fact that it is written in `JAX`, it automatically supports automatic differentiation via `jax.grad` and `jax.jacobian`. 

In this notebook, we show how these partial derivatives can be constructed: for more advanced examples on how to use these gradients for practical applications, see the tutorials on `state_transition_matrix_computation`, `covariance_propagation`, `graident_based_optimization`, `orbit_determination`.

## ⚠️ JAX Version Note

This notebook has been updated for the **JAX implementation** (`dsgp4_jax`).

**Key differences from PyTorch version:**
- Import: `import dsgp4_jax` (instead of `import dsgp4`)
- Tensors: Use `jax.numpy` arrays instead of `torch` tensors
- For automatic differentiation examples, see the [JAX Migration Guide](../../JAX_MIGRATION_GUIDE.md)

The core API remains the same, so most code works as-is!

## ✅ JAX Implementation Complete

This notebook has been fully converted to use **JAX autodiff** instead of PyTorch.

**Key JAX patterns used:**
- `jax.grad()` for scalar-valued functions
- `jax.jacobian()` for vector-valued functions  
- Functional approach (no `.backward()` needed)

All code examples now work with `dsgp4_jax`!

In [1]:
import dsgp4_jax
import jax
import jax.numpy as jnp

We create a TLE object

In [2]:
#as always, first, we create a TLE object:
tle=[]
tle.append('0 COSMOS 2251 DEB')
tle.append('1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996')
tle.append('2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320')
tle = dsgp4_jax.tle.TLE(tle)
print(tle)

TLE(
0 COSMOS 2251 DEB
1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996
2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320
)


Now, as shown in the `tle_propagation` tutorial, we can propagate the TLE. In JAX, we use `jax.jacobian` to compute gradients with respect to the time.

## Partials with respect to time

Let's compute the partials of the $\partial \textrm{SGP4}$ output w.r.t. the propagation times

### Single TLEs

Let's first see the case of single TLEs, propagated at various future times:

In [3]:
# Create random times (no requires_grad needed in JAX)
tsince = jnp.linspace(0.1, 1.0, 10)  # Use fixed times for reproducibility
# Get state
state_teme = dsgp4_jax.propagate(tle, tsince, initialized=False)
print(state_teme)

tensor([[[ 1.3801e+03, -6.9928e+03,  1.9835e+02],
         [ 1.9807e+00,  5.8705e-01,  7.1998e+00]],

        [[ 1.3596e+03, -6.9984e+03,  1.2413e+02],
         [ 1.9962e+00,  5.0767e-01,  7.2017e+00]],

        [[ 1.3946e+03, -6.9883e+03,  2.5111e+02],
         [ 1.9695e+00,  6.4345e-01,  7.1980e+00]],

        [[ 1.3492e+03, -7.0010e+03,  8.6688e+01],
         [ 2.0040e+00,  4.6760e-01,  7.2023e+00]],

        [[ 1.3540e+03, -6.9998e+03,  1.0394e+02],
         [ 2.0004e+00,  4.8606e-01,  7.2020e+00]],

        [[ 1.4152e+03, -6.9811e+03,  3.2657e+02],
         [ 1.9533e+00,  7.2407e-01,  7.1947e+00]],

        [[ 1.4245e+03, -6.9776e+03,  3.6089e+02],
         [ 1.9458e+00,  7.6073e-01,  7.1929e+00]],

        [[ 1.3690e+03, -6.9960e+03,  1.5791e+02],
         [ 1.9892e+00,  5.4380e-01,  7.2009e+00]],

        [[ 1.3816e+03, -6.9924e+03,  2.0371e+02],
         [ 1.9795e+00,  5.9278e-01,  7.1997e+00]],

        [[ 1.3448e+03, -7.0020e+03,  7.0776e+01],
         [ 2.0073e+00,  4.5057e-

Let's now retrieve the partial derivatives of the SGP4 output w.r.t. time.

Since the state is position and velocity (i.e., $[x,y,z,v_x,v_y,v_z]$), these partials will be all the elements of type:
\begin{equation}
\dfrac{d \pmb{x}}{d t}=[\dfrac{dx}{dt}, \dfrac{dy}{dt}, \dfrac{dz}{dt}, \dfrac{d^2x}{dt^2}, \dfrac{d^2y}{dt^2}, \dfrac{d^2z}{dt^2}]^T=[v_x, v_y, v_z, \dfrac{dv_x}{dt}, \dfrac{dv_y}{dt}, \dfrac{dv_z}{dt}]^T
\end{equation}


```{note}
One thing to be careful about is that $\partial\textrm{SGP4}$, mirroring the original $\textrm{SGP4}$, takes the time in minutes, and returns the state in km and km/s. Hence, the derivatives will have dimensions coherent to these, and to return to SI, conversions have to be made.
```

In [4]:
# Compute partial derivatives using JAX grad
# JAX approach: define function for each output component

def propagate_at_times(t_array):
    """Propagate and return full state"""
    return dsgp4_jax.propagate(tle, t_array, initialized=True)

# Compute gradients for each position and velocity component
partial_derivatives = jnp.zeros_like(state_teme)

for i in [0, 1]:  # position (0) and velocity (1)
    for j in [0, 1, 2]:  # x, y, z components
        # Define function that extracts specific component
        def component_fn(t_array):
            state = propagate_at_times(t_array)
            return state[:, i, j]
        
        # Compute Jacobian (gradient w.r.t. each time value)
        jacobian_fn = jax.jacobian(component_fn)
        # Get diagonal (derivative of output[k] w.r.t. input[k])
        jac_full = jacobian_fn(tsince)
        partial_derivatives = partial_derivatives.at[:, i, j].set(jnp.diag(jac_full))

# Print the partial derivatives
print(partial_derivatives)

tensor([[[ 1.1884e+02,  3.5223e+01,  4.3199e+02],
         [-9.1163e-02,  4.6190e-01, -1.3136e-02]],

        [[ 1.1977e+02,  3.0460e+01,  4.3210e+02],
         [-8.9812e-02,  4.6229e-01, -8.2214e-03]],

        [[ 1.1817e+02,  3.8607e+01,  4.3188e+02],
         [-9.2116e-02,  4.6159e-01, -1.6630e-02]],

        [[ 1.2024e+02,  2.8056e+01,  4.3214e+02],
         [-8.9127e-02,  4.6247e-01, -5.7415e-03]],

        [[ 1.2003e+02,  2.9164e+01,  4.3212e+02],
         [-8.9443e-02,  4.6239e-01, -6.8839e-03]],

        [[ 1.1720e+02,  4.3444e+01,  4.3168e+02],
         [-9.3471e-02,  4.6110e-01, -2.1626e-02]],

        [[ 1.1675e+02,  4.5644e+01,  4.3157e+02],
         [-9.4083e-02,  4.6086e-01, -2.3898e-02]],

        [[ 1.1935e+02,  3.2628e+01,  4.3206e+02],
         [-9.0428e-02,  4.6212e-01, -1.0458e-02]],

        [[ 1.1877e+02,  3.5567e+01,  4.3198e+02],
         [-9.1260e-02,  4.6187e-01, -1.3491e-02]],

        [[ 1.2044e+02,  2.7034e+01,  4.3215e+02],
         [-8.8835e-02,  4.6254e-

### Batch TLEs

Let's now see how it works for batch TLEs. The API is basically identical:

In [ ]:
#we load 6 TLEs:
inp_file="""0 PSLV DEB
1 35350U 01049QJ  22068.76869562  .00000911  00000-0  24939-3 0  9998
2 35350  98.6033  64.7516 0074531  99.8340 261.1278 14.48029442457561
0 PSLV DEB *
1 35351U 01049QK  22066.70636923  .00002156  00000-0  63479-3 0  9999
2 35351  98.8179  29.5651 0005211  45.5944 314.5671 14.44732274457505
0 SL-18 DEB
1 35354U 93014BD  22068.76520028  .00021929  00000-0  20751-2 0  9995
2 35354  75.7302 100.7819 0059525 350.7978   9.2117 14.92216400847487
0 SL-18 DEB
1 35359U 93014BJ  22068.55187275  .00025514  00000-0  24908-2 0  9992
2 35359  75.7369 156.1582 0054843  50.5279 310.0745 14.91164684775759
0 SL-18 DEB
1 35360U 93014BK  22068.44021735  .00019061  00000-0  20292-2 0  9992
2 35360  75.7343 127.2487 0071107  32.5913 327.9635 14.86997880798827
0 METEOR 2-17 DEB
1 35364U 88005Y   22067.81503681  .00001147  00000-0  84240-3 0  9995
2 35364  82.5500  92.4124 0018834 303.2489 178.0638 13.94853833332534"""
lines=inp_file.splitlines()
#let's create the TLE objects
tles=[]
for i in range(0,len(lines),3):
    data=[]
    data.append(lines[i])
    data.append(lines[i+1])
    data.append(lines[i+2])
    tles.append(dsgp4_jax.tle.TLE(data))
#we also create 6 random times (fixed for reproducibility):
tsinces = jnp.array([0.23, 0.47, 0.12, 0.89, 0.56, 0.34])

In [8]:
#let's initialize the TLEs:
_,tle_batch=dsgp4_jax.initialize_tle(tles)
#let's propagate the batch:
state_teme = dsgp4_jax.propagate_batch(tle_batch,
                tsinces=tsinces)

Now, let's retrieve the partial of each TLE, at each propagated time, and store them into a Nx2x3 matrix:

In [9]:
# Retrieve partials w.r.t. time for batch
def propagate_batch_at_times(t_array):
    return dsgp4_jax.propagate_batch(tle_batch, tsinces=t_array)

partial_derivatives = jnp.zeros_like(state_teme)

for i in [0, 1]:
    for j in [0, 1, 2]:
        def component_fn(t_array):
            state = propagate_batch_at_times(t_array)
            return state[:, i, j]
        
        jacobian_fn = jax.jacobian(component_fn)
        jac_full = jacobian_fn(tsinces)
        partial_derivatives = partial_derivatives.at[:, i, j].set(jnp.diag(jac_full))

print(partial_derivatives)

tensor([[[ 5.7015e+01, -3.6352e+01,  4.4362e+02],
         [-2.0210e-01, -4.2678e-01, -4.9904e-03]],

        [[ 1.2403e+01, -7.1946e+01,  4.4313e+02],
         [-4.1210e-01, -2.2931e-01, -2.5077e-02]],

        [[-1.0693e+02, -3.9097e+01,  4.4204e+02],
         [ 9.8062e-02, -4.8840e-01, -1.9590e-02]],

        [[-2.3161e+01, -1.1218e+02,  4.4075e+02],
         [ 4.5547e-01, -1.9488e-01, -2.3106e-02]],

        [[-7.2915e+01, -8.9360e+01,  4.4129e+02],
         [ 3.0545e-01, -3.9066e-01, -2.6374e-02]],

        [[ 4.6004e+01, -3.7524e+02, -2.3086e+02],
         [ 3.9414e-02,  2.3749e-01, -3.7830e-01]]])


## Partials with respect to TLE parameters

Let's now tackle the case in which we are interested in the partials of the $\partial\textrm{SGP4}$ output w.r.t. the TLE parameters.

### Single TLEs

We first tackle the case of single TLE, propagated at multiple times:

In this case, we want the Jacobian of the output state, w.r.t. the following TLE parameters $\textrm{TLE}=[n,e,i,\Omega,\omega,M,B^*,\dot{n},\ddot{n}]$, where:

* $n$ is the mean motion (also known as `no_kozai` in the original implementation) [rad/minute]; 
* $e$ is the eccentricity [-]; 
* $i$ is the inclination [rad]; 
* $\Omega$ is the right ascension of the ascending node [rad]; 
* $\omega$ is the argument of perigee [rad];
* $M$ is the mean anomaly [rad];
* $B^*$ is the Bstar parameter [1/earth radii]
* $\dot{n}$ mean motion first derivative [radians/$\textrm{minute}^2$]
* $\ddot{n}$ mean motion second derivative [radians/$\textrm{minute}^2$]

\begin{equation}
\dfrac{\partial \pmb{x}}{\partial \textrm{TLE}}=
\begin{bmatrix}
\frac{\partial x}{\partial B^*} & \frac{\partial x}{\partial \dot{n}} & \frac{\partial x}{\partial \ddot{n}} & \frac{\partial x}{\partial e} & \frac{\partial x}{\partial \omega} & \frac{\partial x}{\partial i} & \frac{\partial x}{\partial M} & \frac{\partial x}{\partial n} & \frac{\partial x}{\partial \Omega} \\
\\
\frac{\partial y}{\partial B^*} & \frac{\partial y}{\partial \dot{n}} & \frac{\partial y}{\partial \ddot{n}} & \frac{\partial y}{\partial e} & \frac{\partial y}{\partial \omega} & \frac{\partial y}{\partial i} & \frac{\partial y}{\partial M} & \frac{\partial y}{\partial n} & \frac{\partial y}{\partial \Omega} \\
\\
\frac{\partial z}{\partial B^*} & \frac{\partial z}{\partial \dot{n}} & \frac{\partial z}{\partial \ddot{n}} & \frac{\partial z}{\partial e} & \frac{\partial z}{\partial \omega} & \frac{\partial z}{\partial i} & \frac{\partial z}{\partial M} & \frac{\partial z}{\partial n} & \frac{\partial z}{\partial \Omega} \\
\\
\frac{\partial v_x}{\partial B^*} & \frac{\partial v_x}{\partial \dot{n}} & \frac{\partial v_x}{\partial \ddot{n}} & \frac{\partial v_x}{\partial e} & \frac{\partial v_x}{\partial \omega} & \frac{\partial v_x}{\partial i} & \frac{\partial v_x}{\partial M} & \frac{\partial v_x}{\partial n} & \frac{\partial v_x}{\partial \Omega} \\
\\
\frac{\partial v_y}{\partial B^*} & \frac{\partial v_y}{\partial \dot{n}} & \frac{\partial v_y}{\partial \ddot{n}} & \frac{\partial v_y}{\partial e} & \frac{\partial v_y}{\partial \omega} & \frac{\partial v_y}{\partial i} & \frac{\partial v_y}{\partial M} & \frac{\partial v_y}{\partial n} & \frac{\partial v_y}{\partial \Omega} \\
\\
\frac{\partial v_z}{\partial B^*} & \frac{\partial v_z}{\partial \dot{n}} & \frac{\partial v_z}{\partial \ddot{n}} & \frac{\partial v_z}{\partial e} & \frac{\partial v_z}{\partial \omega} & \frac{\partial v_z}{\partial i} & \frac{\partial v_z}{\partial M} & \frac{\partial v_z}{\partial n} & \frac{\partial v_z}{\partial \Omega} \\
\\
\frac{\partial \dot{n}}{\partial B^*} & \frac{\partial \dot{n}}{\partial \dot{n}} & \frac{\partial \dot{n}}{\partial \ddot{n}} & \frac{\partial \dot{n}}{\partial e} & \frac{\partial \dot{n}}{\partial \omega} & \frac{\partial \dot{n}}{\partial i} & \frac{\partial \dot{n}}{\partial M} & \frac{\partial \dot{n}}{\partial n} & \frac{\partial \dot{n}}{\partial \Omega} \\
\\
\frac{\partial \ddot{n}}{\partial B^*} & \frac{\partial \ddot{n}}{\partial \dot{n}} & \frac{\partial \ddot{n}}{\partial \ddot{n}} & \frac{\partial \ddot{n}}{\partial e} & \frac{\partial \ddot{n}}{\partial \omega} & \frac{\partial \ddot{n}}{\partial i} & \frac{\partial \ddot{n}}{\partial M} & \frac{\partial \ddot{n}}{\partial n} & \frac{\partial \ddot{n}}{\partial \Omega}
\end{bmatrix}
\end{equation}

In [10]:
#as always, first, we create a TLE object:
tle=[]
tle.append('0 COSMOS 2251 DEB')
tle.append('1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996')
tle.append('2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320')
tle = dsgp4_jax.tle.TLE(tle)
print(tle)
tle_elements=dsgp4_jax.initialize_tle(tle,with_grad=True)

TLE(
0 COSMOS 2251 DEB
1 34454U 93036SX  22068.91971155  .00000319  00000-0  11812-3 0  9996
2 34454  74.0583 280.7094 0037596 327.9100  31.9764 14.35844873683320
)


In [11]:
# Select random times (no torch.rand needed)
tsince = jnp.array([0.1, 0.5, 1.0, 2.0, 5.0, 7.0, 9.0, 10.0, 12.0, 15.0])
# Propagate
state_teme = dsgp4_jax.propagate(tle, tsince)

In [12]:
# Build partial derivatives matrix w.r.t. TLE parameters
# Shape: Nx6x9 (N times, 6 state components, 9 TLE elements)

# First, get the TLE elements we'll differentiate w.r.t.
tle_elements = jnp.array([tle._bstar, tle._ndot, tle._nddot, 
                           tle._ecco, tle._argpo, tle._inclo,
                           tle._mo, tle._no_kozai, tle._nodeo])

# For JAX, we need to create a function that takes TLE elements and returns state
def propagate_with_tle_elements(elements):
    """Propagate with modified TLE elements"""
    whichconst = dsgp4_jax.util.get_gravity_constants("wgs-84")
    temp_tle = dsgp4_jax.tle.TLE([tle.line1, tle.line2])
    
    dsgp4_jax.sgp4init(
        whichconst=whichconst,
        opsmode='i',
        satn=tle.satellite_catalog_number,
        epoch=(tle._jdsatepoch + tle._jdsatepochF) - 2433281.5,
        xbstar=elements[0],
        xndot=elements[1],
        xnddot=elements[2],
        xecco=elements[3],
        xargpo=elements[4],
        xinclo=elements[5],
        xmo=elements[6],
        xno_kozai=elements[7],
        xnodeo=elements[8],
        satellite=temp_tle
    )
    
    return dsgp4_jax.sgp4(temp_tle, tsince)

# Compute Jacobian for all time steps at once
jacobian_fn = jax.jacobian(propagate_with_tle_elements)
full_jacobian = jacobian_fn(tle_elements)

# Reshape to desired format: (N_times, 6_states, 9_elements)
partial_derivatives = jnp.transpose(full_jacobian, (1, 2, 0, 3)).reshape(len(tsince), 6, 9)

print(partial_derivatives)

tensor([[[-6.3133e-05,  0.0000e+00,  0.0000e+00,  9.4655e+02,  1.8975e+03,
          -1.3577e+02,  1.9123e+03, -1.3931e+04,  6.9975e+03],
         [-3.1846e-04,  0.0000e+00,  0.0000e+00,  6.4258e+03,  5.1208e+02,
          -2.9033e+01,  5.0081e+02,  7.4555e+04,  1.3634e+03],
         [-4.2398e-04,  0.0000e+00,  0.0000e+00,  7.4314e+03,  6.8559e+03,
           3.5277e+01,  6.8995e+03,  5.8550e+02,  0.0000e+00],
         [-6.3672e-07,  0.0000e+00,  0.0000e+00,  8.7674e-01, -1.4296e+00,
          -7.0726e+00, -1.4383e+00,  1.0140e+01, -5.2221e-01],
         [-3.8001e-07,  0.0000e+00,  0.0000e+00,  4.5007e+00,  7.3593e+00,
          -1.3399e+00,  7.3815e+00,  5.0480e+00,  1.9934e+00],
         [-2.4335e-06,  0.0000e+00,  0.0000e+00,  5.9475e+00, -1.3037e-01,
           2.0514e+00, -1.4565e-01,  3.8289e+01,  0.0000e+00]],

        [[-1.2694e-04,  0.0000e+00,  0.0000e+00,  9.5994e+02,  1.8746e+03,
          -2.4796e+02,  1.8892e+03, -1.3774e+04,  6.9882e+03],
         [-5.9015e-04,  0.0000e+

### Batch TLEs:

As for the time derivatives, the API stays practically identical:

In [ ]:
#we load 6 TLEs:
inp_file="""0 PSLV DEB
1 35350U 01049QJ  22068.76869562  .00000911  00000-0  24939-3 0  9998
2 35350  98.6033  64.7516 0074531  99.8340 261.1278 14.48029442457561
0 PSLV DEB *
1 35351U 01049QK  22066.70636923  .00002156  00000-0  63479-3 0  9999
2 35351  98.8179  29.5651 0005211  45.5944 314.5671 14.44732274457505
0 SL-18 DEB
1 35354U 93014BD  22068.76520028  .00021929  00000-0  20751-2 0  9995
2 35354  75.7302 100.7819 0059525 350.7978   9.2117 14.92216400847487
0 SL-18 DEB
1 35359U 93014BJ  22068.55187275  .00025514  00000-0  24908-2 0  9992
2 35359  75.7369 156.1582 0054843  50.5279 310.0745 14.91164684775759
0 SL-18 DEB
1 35360U 93014BK  22068.44021735  .00019061  00000-0  20292-2 0  9992
2 35360  75.7343 127.2487 0071107  32.5913 327.9635 14.86997880798827
0 METEOR 2-17 DEB
1 35364U 88005Y   22067.81503681  .00001147  00000-0  84240-3 0  9995
2 35364  82.5500  92.4124 0018834 303.2489 178.0638 13.94853833332534"""
lines=inp_file.splitlines()
#let's create the TLE objects
tles=[]
for i in range(0,len(lines),3):
    data=[]
    data.append(lines[i])
    data.append(lines[i+1])
    data.append(lines[i+2])
    tles.append(dsgp4_jax.tle.TLE(data))

#we create 6 fixed times (for reproducibility - use same values to compare with other runs):
# NOTE: The PyTorch version used torch.rand((6,)) which generates different values each run.
# To get identical numerical results, use the exact same time values here.
tsinces = jnp.array([0.23, 0.47, 0.12, 0.89, 0.56, 0.34])

In [15]:
#let's now initialize the TLEs, activating the gradient tracking for the TLE parameters:
tle_elements,tle_batch=dsgp4_jax.initialize_tle(tles,with_grad=True)


In [17]:
#let's now propagate the batch of TLEs:
state_teme = dsgp4_jax.propagate_batch(tle_batch,tsinces)

Finally, we can build the matrix that contains the partial of the SGP4 output w.r.t. the TLE parameters, for each TLE:

In [ ]:
# Build partial derivatives matrix w.r.t. TLE parameters for batch
# Shape: Nx6x9 (N TLEs, 6 state components, 9 TLE elements)
# Uses reverse-mode autodiff (like PyTorch's backward) for efficiency

def propagate_single_tle_with_elements(elements, tle_template, tsince_val):
    """Propagate a single TLE with modified elements"""
    whichconst = dsgp4_jax.util.get_gravity_constants("wgs-84")
    temp_tle = dsgp4_jax.tle.TLE([tle_template.line1, tle_template.line2])
    
    dsgp4_jax.sgp4init(
        whichconst=whichconst,
        opsmode='i',
        satn=tle_template.satellite_catalog_number,
        epoch=(tle_template._jdsatepoch + tle_template._jdsatepochF) - 2433281.5,
        xbstar=elements[0],
        xndot=elements[1],
        xnddot=elements[2],
        xecco=elements[3],
        xargpo=elements[4],
        xinclo=elements[5],
        xmo=elements[6],
        xno_kozai=elements[7],
        xnodeo=elements[8],
        satellite=temp_tle
    )
    
    return dsgp4_jax.sgp4(temp_tle, jnp.array([[tsince_val]])).flatten()

def compute_single_output(elements, tle_template, tsince_val, output_idx):
    """Propagate and return a single output component (for reverse-mode grad)"""
    state = propagate_single_tle_with_elements(elements, tle_template, tsince_val)
    return state[output_idx]

# Compute Jacobians using reverse-mode autodiff (more efficient, matches PyTorch)
partial_derivatives = []
for k, (tle_obj, tsince_val) in enumerate(zip(tles, tsinces)):
    tle_elements_k = jnp.array([tle_obj._bstar, tle_obj._ndot, tle_obj._nddot,
                                 tle_obj._ecco, tle_obj._argpo, tle_obj._inclo,
                                 tle_obj._mo, tle_obj._no_kozai, tle_obj._nodeo])
    
    # Compute gradient for each output component (reverse-mode, like PyTorch backward)
    jac_rows = []
    for i in range(6):
        grad_fn = jax.grad(lambda el: compute_single_output(el, tle_obj, float(tsince_val), i))
        grad_vec = grad_fn(tle_elements_k)
        jac_rows.append(grad_vec)
    
    jac = jnp.stack(jac_rows)
    partial_derivatives.append(jac)

partial_derivatives = jnp.stack(partial_derivatives)
print(partial_derivatives)